<a href="https://colab.research.google.com/github/Hari1128/Hackathon/blob/main/phenome.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install phonemizer



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.8/103.8 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.7/60.7 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.4/213.4 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.2/587.2 kB 25.1 MB/s eta 0:00:00


In [2]:
!pip install indic-transliteration


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.6/159.6 kB 9.4 MB/s eta 0:00:00


In [3]:
import re
import difflib
from phonemizer import phonemize
from indic_transliteration import sanscript
from indic_transliteration.sanscript import transliterate


In [4]:
!apt-get install -y espeak


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  espeak-data libespeak1 libportaudio2 libsonic0
The following NEW packages will be installed:
  espeak espeak-data libespeak1 libportaudio2 libsonic0
0 upgraded, 5 newly installed, 0 to remove and 1 not upgraded.
Need to get 1,382 kB of archives.
After this operation, 3,178 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libportaudio2 amd64 19.6.0-1.1 [65.3 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libsonic0 amd64 0.2.0-11build1 [10.3 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 espeak-data amd64 1.48.15+dfsg-3 [1,085 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libespeak1 amd64 1.48.15+dfsg-3 [156 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy/universe amd64 espeak amd64 1.48.15+dfsg-3 [64.2 kB]
Fetched 1,382 kB in 1s (1,783 k

In [5]:
!sudo apt-get install -y espeak


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
espeak is already the newest version (1.48.15+dfsg-3).
0 upgraded, 0 newly installed, 0 to remove and 1 not upgraded.


In [6]:
!pip install phonemizer indic-transliteration


In [7]:
import re
import difflib
from phonemizer import phonemize
from indic_transliteration import sanscript
from indic_transliteration.sanscript import transliterate


In [11]:
from google.colab import files
uploaded = files.upload()

audio_path = list(uploaded.keys())[0]
print("Uploaded:", audio_path)


Saving p1.wav.wav to p1.wav.wav
Uploaded: p1.wav.wav


In [12]:
!pip install -U openai-whisper


  Using cached openai_whisper-20250625.tar.gz (803 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.5/170.5 MB 7.1 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20250625-py3-none-any.whl size=803979 sha256=36451b816b1923be51b3fcd71decc73c5e407845c6ccf13da07b3280d37d2d74
  Stored in directory: /root/.cache/pip/wheels/61/d2/20/09ec9bef734d126cba375b15898010b6cc28578d8afdde5869
Successfully built openai-whisper


In [13]:
import whisper

model = whisper.load_model("small")

result = model.transcribe(audio_path)
spoken_text = result["text"].lower()
language = result["language"]   # 'en' or 'ta'

print("Detected language:", language)
print("Spoken text:", spoken_text)


100%|████████████████████████████████████████| 461M/461M [00:03<00:00, 143MiB/s]
/usr/local/lib/python3.12/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Detected language: en
Spoken text:  i need to go to hospital.


In [14]:
EXPECTED_EN = "i need to go to the hospital today"
EXPECTED_TA = "எனக்கு இன்று மருத்துவமனை செல்ல வேண்டும்"

expected_text = EXPECTED_TA if language == "ta" else EXPECTED_EN

print("Expected text:", expected_text)


Expected text: i need to go to the hospital today


In [15]:
def normalize(text):
    text = text.lower()
    text = re.sub(r"[^\w\s]", "", text)
    return re.sub(r"\s+", " ", text).strip()


In [16]:
def english_phonemes(text):
    return phonemize(
        text,
        language="en-us",
        backend="espeak",
        strip=True
    ).split()

def tamil_phonemes(text):
    roman = transliterate(text, sanscript.TAMIL, sanscript.ITRANS)
    return roman_phonemes(roman)

def roman_phonemes(roman):
    out, i = [], 0
    combos = ["aa","ii","uu","ee","oo","ai","au","zh","ng","ny","th","dh","sh"]
    while i < len(roman):
        if roman[i:i+2] in combos:
            out.append(roman[i:i+2])
            i += 2
        else:
            out.append(roman[i])
            i += 1
    return out


In [17]:
VOWELS = {"a","aa","i","ii","u","uu","e","ee","o","oo","ai","au"}

def ptype(p):
    return "vowel" if p in VOWELS else "consonant"


def phoneme_weight(p):
    return 1.2 if ptype(p) == "vowel" else 1.0


def analyze_phonemes(expected, spoken, lang):
    expected = normalize(expected)
    spoken = normalize(spoken)

    exp_ph = tamil_phonemes(expected) if lang == "ta" else english_phonemes(expected)
    spk_ph = tamil_phonemes(spoken) if lang == "ta" else english_phonemes(spoken)

    matcher = difflib.SequenceMatcher(None, exp_ph, spk_ph)

    subs, omis, dist = [], [], []
    total_weight = 0
    error_weight = 0

    # total expected phoneme weight
    for p in exp_ph:
        total_weight += phoneme_weight(p)

    for tag, i1, i2, j1, j2 in matcher.get_opcodes():

        if tag == "replace":   # substitution
            for e, s in zip(exp_ph[i1:i2], spk_ph[j1:j2]):
                w = phoneme_weight(e)
                error_weight += w
                subs.append({
                    "expected": e,
                    "spoken": s,
                    "type": ptype(e),
                    "penalty": w
                })

        elif tag == "delete":  # omission
            for e in exp_ph[i1:i2]:
                w = phoneme_weight(e)
                error_weight += w
                omis.append({
                    "phoneme": e,
                    "type": ptype(e),
                    "penalty": w
                })

        elif tag == "insert":  # insertion / distortion
            for s in spk_ph[j1:j2]:
                w = phoneme_weight(s)
                error_weight += w * 0.5   # insertion penalized less
                dist.append({
                    "phoneme": s,
                    "type": ptype(s),
                    "penalty": w * 0.5
                })

    # phoneme score
    phoneme_score = max(
        0,
        round((1 - error_weight / total_weight) * 100, 2)
    )

    return subs, omis, dist, phoneme_score


In [18]:
subs, omis, dist, phoneme_score = analyze_phonemes(
    expected_text,
    spoken_text,
    language
)


In [19]:
print("\nSUBSTITUTIONS:")
for s in subs:
    print(f"{s['expected']} → {s['spoken']} ({s['type']}) penalty={s['penalty']}")

print("\nOMISSIONS:")
for o in omis:
    print(f"{o['phoneme']} ({o['type']}) penalty={o['penalty']}")

print("\nDISTORTIONS:")
for d in dist:
    print(f"{d['phoneme']} ({d['type']}) penalty={d['penalty']}")

print("\nPHONEME SCORE:", phoneme_score, "%")



SUBSTITUTIONS:

OMISSIONS:
ðə (consonant) penalty=1.0
tədeɪ (consonant) penalty=1.0

DISTORTIONS:

PHONEME SCORE: 75.0 %
